In [1]:
import argparse
import logging
import db.mariadb as mariadb
import os
import MySQLdb
import keras


from requests_futures.sessions import FuturesSession

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(format='%(asctime)s-%(levelname)s-%(name)s - %(message)s')


model = keras.applications.resnet50.ResNet50(include_top=False, pooling='avg')


Using TensorFlow backend.
2019-05-21 12:42:59,960-WARNING-tensorflow - From /usr/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [2]:
import keras
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from PIL.Image import DecompressionBombError
import numpy as np

# Change according to your database setup (these are the defaults, see .env file)

con = mariadb.get_connection("127.0.0.1", 3308, "image_processing", "user", "user_pw")

files_batch = mariadb.get_files_data(0, 800000, con)

con.close()

# Use every 100th image of the 800000
files_batch = files_batch
features_train = []
features_test = []

features_train_info = []
features_test_info = []

# Create training and test features
counter = 1
for (img_id, name, path, url) in files_batch:
    try:
        img = keras.preprocessing.image.load_img(path, target_size=(224, 224), grayscale=True)
        # print(type(img))
        img_data = image.img_to_array(img)
        img_data = np.repeat(img_data, 3, axis=2)
        # print(type(img_data))
        # print(img_data.shape)
        img_data = np.expand_dims(img_data, axis=0)
        # print(img_data.shape)
        img_data = preprocess_input(img_data)
        # print(img_data.shape)
        res_net_feature = model.predict(img_data)
        # print(res_net_feature.shape)
        res_net_feature = np.array(res_net_feature).flatten()
        # print(res_net_feature.shape)

        if counter % 3 == 0:
            features_test.append(res_net_feature)
            features_test_info.append((img_id, name, path, url))
        else:
            features_train.append(res_net_feature)
            features_train_info.append((img_id, name, path, url))

        counter += 1

    except OSError as e:
        print(e)
    except DecompressionBombError as e:
        print(e)
        print(path)

features_train = np.array(features_train)
features_test = np.array(features_test)

print(features_train.shape)
print(len(features_train_info))

print(features_test.shape)
print(len(features_test_info))

/home/shohl/.local/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:98: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


Image size (207616860 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.
/mnt/idai_cloud/dai-rom-fotothek-2007/Bestand-D-DAI-ROM-2007.5572.JPG
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0636.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0406.JPG'
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0575/D-DAI-ROM-42.507R_56426,01.jpg'
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0575/D-DAI-ROM-56.998_002922504,02.jpg'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0127.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0442.JPG'


/home/shohl/.local/lib/python3.7/site-packages/PIL/Image.py:2618: DecompressionBombWarning: Image size (124167394 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


cannot identify image file '/mnt/idai_cloud/FADatenbankabb0577/D-DAI-ROM-71.623R_14505.jpg'


/home/shohl/.local/lib/python3.7/site-packages/PIL/Image.py:2618: DecompressionBombWarning: Image size (105211691 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


...

cannot identify image file '/mnt/idai_cloud/FADatenbankabb0576/D-DAI-ROM-62.1474_130217.jpg'


/home/shohl/.local/lib/python3.7/site-packages/PIL/Image.py:2618: DecompressionBombWarning: Image size (136193850 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/shohl/.local/lib/python3.7/site-packages/PIL/Image.py:2618: DecompressionBombWarning: Image size (97257268 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0270.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0672.JPG'


/home/shohl/.local/lib/python3.7/site-packages/PIL/Image.py:2618: DecompressionBombWarning: Image size (144643345 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0188.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0095.JPG'
[Errno 2] No such file or directory: '/mnt/idai_cloud/FADatenbankabb0532/D-DAI-ROM-58.2011A_80541,50.jpg'
[Errno 2] No such file or directory: '/mnt/idai_cloud/FADatenbankabb0024/FA4268-1_76603,3.jpg'
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0700d/Mal000-0_0002624104,03.jpg'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0663.JPG'
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0577/D-DAI-ROM-71.623R_402540.jpg'
(9988, 2048)
9988
(4993, 2048)
4993


In [3]:
# We need to scale the features to values between 0 and 1, because the sigmoid layer (last autoencoder layer) 
# produces values in that range. Without scaling the original values down, the autoencoder won't be able to 
# reproduce input values > 1.

print(np.max(features_train))
print(np.max(features_test))

if(np.max(features_train) > np.max(features_test)):
    features_train_scaled = features_train / np.max(features_train)
    features_test_scaled = features_test / np.max(features_train)
else:
    features_train_scaled = features_train / np.max(features_test)
    features_test_scaled = features_test / np.max(features_test)

print(np.max(features_train_scaled))
print(np.max(features_test_scaled))

25.173534
26.796541
0.9394322
1.0


In [4]:
# Initialize and run autoencoder

from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam, Adadelta
from keras import regularizers
import keras.backend as K

def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

# Stop if there has been no improvement for 100 epochs
es = keras.callbacks.EarlyStopping(verbose=1, patience=100)
# Save best model while training
mc = keras.callbacks.ModelCheckpoint('model/best_model.h5')

callbacks_list = [es, mc]

m = Sequential()

m.add(Dense(512, activation='elu', input_shape=(features_train_scaled.shape[1],)))
m.add(Dense(32, activation='linear', name="bottleneck"))
m.add(Dense(512, activation='elu'))
m.add(Dense(features_train.shape[1], activation='sigmoid'))

m.compile(loss=euclidean_distance_loss, optimizer=Adadelta())

history = m.fit(features_train_scaled, features_train_scaled, batch_size=128, epochs=100000, verbose=1,
                validation_data=(features_test_scaled, features_test_scaled), callbacks=callbacks_list)


2019-05-21 17:50:49,376-WARNING-tensorflow - From /usr/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Train on 9988 samples, validate on 4993 samples
Epoch 1/100000
9988/9988 [==============================] - 9s 866us/step - loss: 3.4968 - val_loss: 1.2978
Epoch 2/100000
9988/9988 [==============================] - 7s 689us/step - loss: 1.2242 - val_loss: 1.2955
Epoch 3/100000
9988/9988 [==============================] - 8s 772us/step - loss: 1.2038 - val_loss: 1.2719
Epoch 4/100000
9988/9988 [==============================] - 10s 1ms/step - loss: 1.1903 - val_loss: 1.2545
Epoch 5/100000
9988/9988 [==============================] - 14s 1ms/step - loss: 1.1788 - val_loss: 1.2430
Epoch 6/100000
9988/9988 [==============================] - 8s 772us/step - loss: 1.1620 - val_loss: 1.2600
Epoch 7/100000
9988/9988 [==============================] - 7s 713us/step - loss: 1.1419 - val_loss: 1.3862
Epoch 8/100000
9988/9988 [==============================] - 7s 662us/step - loss: 1.1163 - val_loss: 1.2140
Epoch 9/100000
9988/9988 [==============================] - 7s 695us/step - loss: 1.0928 -

9988/9988 [==============================] - 7s 698us/step - loss: 0.7307 - val_loss: 0.7812
Epoch 77/100000
9988/9988 [==============================] - 7s 697us/step - loss: 0.7271 - val_loss: 0.7505
Epoch 78/100000
9988/9988 [==============================] - 8s 794us/step - loss: 0.7257 - val_loss: 0.7566
Epoch 79/100000
9988/9988 [==============================] - 7s 689us/step - loss: 0.7236 - val_loss: 0.7538
Epoch 80/100000
9988/9988 [==============================] - 7s 698us/step - loss: 0.7222 - val_loss: 0.7607
Epoch 81/100000
9988/9988 [==============================] - 7s 695us/step - loss: 0.7213 - val_loss: 0.7758
Epoch 82/100000
9988/9988 [==============================] - 7s 742us/step - loss: 0.7192 - val_loss: 0.7841
Epoch 83/100000
9988/9988 [==============================] - 7s 708us/step - loss: 0.7145 - val_loss: 0.7529
Epoch 84/100000
9988/9988 [==============================] - 7s 684us/step - loss: 0.7137 - val_loss: 0.7817
Epoch 85/100000
9988/9988 [========

Epoch 151/100000
9988/9988 [==============================] - 6s 614us/step - loss: 0.6389 - val_loss: 0.6772
Epoch 152/100000
9988/9988 [==============================] - 6s 615us/step - loss: 0.6381 - val_loss: 0.6901
Epoch 153/100000
9988/9988 [==============================] - 6s 613us/step - loss: 0.6386 - val_loss: 0.7034
Epoch 154/100000
9988/9988 [==============================] - 6s 613us/step - loss: 0.6366 - val_loss: 0.6938
Epoch 155/100000
9988/9988 [==============================] - 6s 614us/step - loss: 0.6370 - val_loss: 0.6877
Epoch 156/100000
9988/9988 [==============================] - 6s 634us/step - loss: 0.6358 - val_loss: 0.6855
Epoch 157/100000
9988/9988 [==============================] - 6s 615us/step - loss: 0.6359 - val_loss: 0.6705
Epoch 158/100000
9988/9988 [==============================] - 6s 613us/step - loss: 0.6340 - val_loss: 0.6862
Epoch 159/100000
9988/9988 [==============================] - 6s 625us/step - loss: 0.6346 - val_loss: 0.6782
Epoch 160/

9988/9988 [==============================] - 6s 618us/step - loss: 0.6064 - val_loss: 0.6664
Epoch 226/100000
9988/9988 [==============================] - 6s 618us/step - loss: 0.6057 - val_loss: 0.6658
Epoch 227/100000
9988/9988 [==============================] - 6s 614us/step - loss: 0.6055 - val_loss: 0.6688
Epoch 228/100000
9988/9988 [==============================] - 6s 618us/step - loss: 0.6056 - val_loss: 0.6756
Epoch 229/100000
9988/9988 [==============================] - 6s 618us/step - loss: 0.6053 - val_loss: 0.6619
Epoch 230/100000
9988/9988 [==============================] - 6s 612us/step - loss: 0.6042 - val_loss: 0.6623
Epoch 231/100000
9988/9988 [==============================] - 6s 631us/step - loss: 0.6042 - val_loss: 0.6659
Epoch 232/100000
9988/9988 [==============================] - 6s 626us/step - loss: 0.6052 - val_loss: 0.6686
Epoch 233/100000
9988/9988 [==============================] - 6s 619us/step - loss: 0.6037 - val_loss: 0.6593
Epoch 234/100000
9988/9988 

9988/9988 [==============================] - 6s 615us/step - loss: 0.5883 - val_loss: 0.6656
Epoch 300/100000
9988/9988 [==============================] - 6s 616us/step - loss: 0.5885 - val_loss: 0.6585
Epoch 301/100000
9988/9988 [==============================] - 6s 614us/step - loss: 0.5879 - val_loss: 0.6742
Epoch 302/100000
9988/9988 [==============================] - 6s 618us/step - loss: 0.5867 - val_loss: 0.6546
Epoch 303/100000
9988/9988 [==============================] - 6s 619us/step - loss: 0.5858 - val_loss: 0.6556
Epoch 304/100000
9988/9988 [==============================] - 6s 618us/step - loss: 0.5855 - val_loss: 0.6485
Epoch 305/100000
9988/9988 [==============================] - 6s 618us/step - loss: 0.5859 - val_loss: 0.6581
Epoch 306/100000
9988/9988 [==============================] - 6s 616us/step - loss: 0.5867 - val_loss: 0.6594
Epoch 307/100000
9988/9988 [==============================] - 6s 621us/step - loss: 0.5860 - val_loss: 0.6634
Epoch 308/100000
9988/9988 

9988/9988 [==============================] - 6s 617us/step - loss: 0.5751 - val_loss: 0.6564
Epoch 374/100000
9988/9988 [==============================] - 6s 616us/step - loss: 0.5754 - val_loss: 0.6495
Epoch 375/100000
9988/9988 [==============================] - 6s 615us/step - loss: 0.5751 - val_loss: 0.6572
Epoch 376/100000
9988/9988 [==============================] - 6s 616us/step - loss: 0.5746 - val_loss: 0.6492
Epoch 377/100000
9988/9988 [==============================] - 6s 616us/step - loss: 0.5755 - val_loss: 0.6508
Epoch 378/100000
9988/9988 [==============================] - 6s 615us/step - loss: 0.5750 - val_loss: 0.6594
Epoch 379/100000
9988/9988 [==============================] - 6s 613us/step - loss: 0.5752 - val_loss: 0.6488
Epoch 380/100000
9988/9988 [==============================] - 6s 617us/step - loss: 0.5747 - val_loss: 0.6510
Epoch 381/100000
9988/9988 [==============================] - 6s 616us/step - loss: 0.5737 - val_loss: 0.6604
Epoch 382/100000
9988/9988 

9988/9988 [==============================] - 6s 626us/step - loss: 0.5659 - val_loss: 0.6465
Epoch 448/100000
9988/9988 [==============================] - 6s 622us/step - loss: 0.5663 - val_loss: 0.6480
Epoch 449/100000
9988/9988 [==============================] - 6s 615us/step - loss: 0.5667 - val_loss: 0.6564
Epoch 450/100000
9988/9988 [==============================] - 6s 617us/step - loss: 0.5655 - val_loss: 0.6534
Epoch 451/100000
9988/9988 [==============================] - 6s 619us/step - loss: 0.5658 - val_loss: 0.6530
Epoch 452/100000
9988/9988 [==============================] - 6s 616us/step - loss: 0.5649 - val_loss: 0.6549
Epoch 453/100000
9988/9988 [==============================] - 6s 616us/step - loss: 0.5661 - val_loss: 0.6498
Epoch 454/100000
9988/9988 [==============================] - 6s 615us/step - loss: 0.5665 - val_loss: 0.6538
Epoch 455/100000
9988/9988 [==============================] - 6s 620us/step - loss: 0.5651 - val_loss: 0.6565
Epoch 456/100000
9988/9988 

9988/9988 [==============================] - 6s 616us/step - loss: 0.5606 - val_loss: 0.6587
Epoch 522/100000
9988/9988 [==============================] - 6s 616us/step - loss: 0.5596 - val_loss: 0.6445
Epoch 523/100000
9988/9988 [==============================] - 6s 619us/step - loss: 0.5597 - val_loss: 0.6613
Epoch 524/100000
9988/9988 [==============================] - 6s 621us/step - loss: 0.5603 - val_loss: 0.6521
Epoch 525/100000
9988/9988 [==============================] - 6s 623us/step - loss: 0.5594 - val_loss: 0.6667
Epoch 526/100000
9988/9988 [==============================] - 6s 617us/step - loss: 0.5594 - val_loss: 0.6557
Epoch 527/100000
9988/9988 [==============================] - 6s 615us/step - loss: 0.5596 - val_loss: 0.6494
Epoch 528/100000
9988/9988 [==============================] - 6s 618us/step - loss: 0.5591 - val_loss: 0.6572
Epoch 529/100000
9988/9988 [==============================] - 6s 614us/step - loss: 0.5594 - val_loss: 0.6516
Epoch 530/100000
9988/9988 

In [5]:
from keras.models import load_model
best_model = load_model('model/best_model.h5', custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
encoder = Model(best_model.input, best_model.get_layer('bottleneck').output)
encoder.save('model/encoder.h5')
enc = encoder.predict(features_train_scaled) # returns the encoded values (32 floats instead of 2048)


In [6]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=100)

result = neigh.fit(features_train_scaled)
neighbours = result.kneighbors()

In [7]:
result_enc = neigh.fit(enc)
neighbours_enc = result_enc.kneighbors()

In [8]:
enc_r = best_model.predict(features_train_scaled)      # reconstruction, 2048 -> 32 -> 2048

In [9]:
result_enc_r = neigh.fit(enc_r)
neighbours_enc_r = result_enc.kneighbors()

In [10]:
np.min(features_train_scaled)

0.0

In [11]:
np.min(features_test_scaled)

0.0

In [12]:
np.max(features_train_scaled)

0.9394322

In [14]:
np.max(features_test_scaled)

1.0

In [16]:
print(neighbours[1][0])
print(neighbours_enc_r[1][0])
print(neighbours_enc[1][0])

[7660 1670 9953 2328 4754 5231 3933 3615 5016 8144  845 3968 2536 3195
 6196 4097  718 7788 5408 2647  518 5040 4864 4575 1770 9056 8033 7478
 4951 2166 4191 6421  517 1543 8813 5359 4277 8095 7013 3690 5636 7361
 8172 7560 6996 5918 9501 7606 3529 2585  250 8639  862 3559 8492 4168
 4758 4274 4066 5039 6448 3983 4761 6428 1007 5149 3798 5564 3012 1772
 1371 8543 1615 4985 2898 6852 6004 9280 7551 2933 3672 1213 9048 2102
 8416 1662 8217 7310 8392 8613 5421 9878 6746 8466 7471 2660 6458 1422
 9876 4102]
[7660 1670 5231 9953 3615 2328 4754 8144 2536 3195 3968 5016 3933 4097
  718 7788 4191  845 4575 7478 5040 5408 6421 1770 6196  250 1543 2647
 5636  518 3690 9056 4864 4951 5359 5918  862 3559 2166 2585 7361 8172
 3983 4758 6996 8033 7551  517 8813 4274 7013 9501 5564 6448 4066 5149
 4277 3529 1007 7606 8095 4761 7560 1371 1772 7471 8639 3012 9185 8492
 4985 1662 1213 5039 4168 8854 2119  655 4102 6470 2898 3798 9280 8613
 2933 6126 6428 9048 1615 4818 3353 8416 7649 3054  343 8217 8392

In [18]:
print(neighbours[1][1])
print(neighbours_enc_r[1][1])
print(neighbours_enc[1][1])

[7369 1220 8549 3518 4840 9905 1127 2431 1836 7472 3556 4381 4379 2859
 1296 6360 5743 9095 7863  916 7390 3367 7537 9562 2938 1884 8232 3897
 2253 9584 2817  818 5492 2683 7422 3298 2755 3946 1169 9195 2978 1903
  600 3353 3138 2710 9291 1420 3478 3786 3760 6420 2625 4990 4449 3039
 8558 5125 8759 6910  463 1154  692 5917 1634 2943 6421 9433 7685 5114
 4521 8808 3590 3040 2171  417 1864 7218 5497 9812 1994 7189 1515 9888
 7706 1499 4846 3637 8560 8102 2428 2774 3233 2195 4960 8126 1874 7505
 8090 1233]
[1220 7369 9562 2431 3518 4379 4840 1836 5492 7246 6360 2253 8141 7252
 8549  916 3556 1127 1169 9905 2978 1296 2710 9195 9799 7471 8232 2859
 4381 2943 7472 3897 9095 8090 3353 3298 5114 7390  600 2938 7863 1233
 8558 3138 3039  847 9584 3946 9888 3760 4990 6757 5743 4960 8126 9433
 5502 1903 1634 5125 1192 3996 1420 1019 6420 3367 5342 1635 1154 4521
 1962 4449 5709 3478 1884 9566 1951 2625 5550 1152 2774 1592 4936 6746
 1864  417 1952 5917 8787 1168 2084 8759 2817 6421 2671 7927 3786

In [20]:
print(neighbours[1][2])
print(neighbours_enc_r[1][2])
print(neighbours_enc[1][2])

[5659 3866 4539 9007 3156 5426 8254 7416 2810 7170 8665 4592 7607 9120
 2104 3907 9001 5989 4673  809 9010 2667 3562  550 9852 4952 9353 4845
 6625 6338 7960 7815 6270 4117 3190 7049 5004 7508 8122 9643 5705 9979
 2768 5615 1129 3729 5851 9577 6418 8771 8180 5091 2869 5822  851 3390
 4989 3643 2922 2180 4755  526 2306 5258 8717 7316 9763  853 7053 7792
 7662 1940 5715 1921 9533 3685 6986 7652 7791 8501 9889 7991 6579 8076
 7967 9397  812 2031 7604 7720  512 8862 7256 2341 2261 6359 6807 4944
 1207 8165]
[9007  550 5659 1129 4539 7416 5426 9577  809 9001 3156 3866 6338 9010
 5989 8254 4592 7607 2104 2810 4952 9852 3907 8180 6625 2869 7049 3729
 8665  853 2306 3562 2922 3190 6270 7508 2667 9120 3643 5615 1787 8717
 7815 9353 8076 5237 7610 7170 9979  851 5091 8565 7662 5469 1921 7960
 8496 1940 2180 3390 4845 4117 5258 2391   23 5004 6807 1818 4673 7316
 5851 5241 7792  512 7967 8122 4989 8771 2031 4755 9763 1810  494 5272
  660 5014 2768 1466 7616 9193  812 5368 1207  526 5571 4809 5715

In [22]:
enc.shape

(9988, 32)

In [24]:
features_train_scaled.shape

(9988, 2048)

In [26]:
import math


def compare_list_order(list_a, list_b):
    error = 0
    
    for a_index, a_content in enumerate(list_a):
        try:
            b_index = list_b.index(a_content)
            error += math.fabs(a_index - b_index)
        except ValueError:
            error += len(list_a)
    return error / len(list_a)


overall_error = 0

for image_idx, image_neighbours in enumerate(neighbours[1]):
    overall_error += compare_list_order(list(image_neighbours), list(neighbours_enc[1][image_idx]))      

print(overall_error / (len(neighbours[1])))
    

47.64767020424492
